## Using Python to do Technical Analysis: Find Weighted Average Minimum Price of a Stock
### The goal is to find a Weighted-Average Support Level (i.e. a floor price) across multiple time periods

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta

### Fetch historical stock data from yfinance and place it in a dataframe

In [2]:
# Fetch historical stock data from yahoo based on user inputs. Default Interval is 1 day (can be 1d, 1wk, 1mo, 3mo, etc)
def fetch_stock_data(symbol, years, interval):
    stock_data = (
        yf.Ticker(symbol).history(
            start=datetime.now() - timedelta(days=365 * years), 
            end=datetime.now(), 
            interval=interval
        )
        .reset_index())  # Adds date as a column

    return pd.DataFrame(stock_data)

# Display the full data for the last 100 years
fetch_stock_data('pfe', 100, '1d')

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,1972-06-01 00:00:00-04:00,0.148762,0.151051,0.148762,0.151051,2458771,0.0,0.0
1,1972-06-02 00:00:00-04:00,0.151051,0.151509,0.148763,0.149220,1613885,0.0,0.0
2,1972-06-05 00:00:00-04:00,0.149220,0.149678,0.147847,0.148762,2585251,0.0,0.0
3,1972-06-06 00:00:00-04:00,0.148762,0.152882,0.148305,0.151967,2347469,0.0,0.0
4,1972-06-07 00:00:00-04:00,0.151967,0.151967,0.149678,0.151967,1032077,0.0,0.0
...,...,...,...,...,...,...,...,...
13438,2025-09-22 00:00:00-04:00,24.280001,24.799999,24.030001,24.040001,67863300,0.0,0.0
13439,2025-09-23 00:00:00-04:00,24.100000,24.360001,24.049999,24.129999,38955200,0.0,0.0
13440,2025-09-24 00:00:00-04:00,24.170000,24.180000,23.980000,24.090000,39669400,0.0,0.0
13441,2025-09-25 00:00:00-04:00,24.090000,24.150000,23.580000,23.600000,56738300,0.0,0.0


### Find a stock's minimum close price across multiple time intervals, then average them according to weights

| Time Period | Weight |
|----------|--------|
| 1 Month  | 20%    |
| 3 Months | 20%    |
| 6 Months | 15%    |
| 1 Year   | 15%    |
| 2 Years  | 10%    |
| 3 Years  | 10%    |
| 5 Years  | 5%     |
| 10 Years | 5%     |

In [11]:
# The idea is to predict the 'floor' for selling put options, with heavier weight on 
# more recent price movements while not ignoring long-term trends. 

def wgt_avg_min_close_price(symbol, periods, weights):
        
    now = datetime.now()
    
    mins = []  # To hold weighted minimums

    # Fetch 10 years of data
    all_data = fetch_stock_data(symbol, 10, '1d')
    
    # Convert to datetime and remove timezone
    all_data['Date'] = pd.to_datetime(all_data['Date']).dt.tz_localize(None)

    # For each period, calculate the cutoff date, filter data from that date onward,
    # then find the minimum close price in that slice and append the weighted value to mins
    for period, weight in zip(periods, weights):
        cutoff = now - timedelta(days=int(period * 365))
        sliced = all_data[all_data['Date'] >= cutoff]
        
        if not sliced.empty:
            min_close = sliced['Close'].min()
            mins.append(min_close * weight)
            
            # Uncomment the following line if you'd like to see the minimum value for each period
            #print(f"{round(period,2)}-Year Min: ${min_close:.2f}")  
    return sum(mins)

### Define User Inputs

In [4]:
my_symbol = "PFE"

# Time periods in years and corresponding weights
periods = [1/12, 1/4, 1/2, 1, 2, 3, 5, 10]  
weights = [0.2, 0.2, 0.15, 0.15, 0.1, 0.1, 0.05, 0.05]

wgt_avg_min = wgt_avg_min_close_price(my_symbol, periods, weights)
    
print(f"\nWgt Avg Min Close Price for {my_symbol} Stock: ${wgt_avg_min:.2f}")

0.08-Year Min: $23.60
0.25-Year Min: $23.29
0.5-Year Min: $20.83
1-Year Min: $20.83
2-Year Min: $20.83
3-Year Min: $20.83
5-Year Min: $20.83
10-Year Min: $18.10

Wgt Avg Min Close Price for PFE Stock: $21.74


### We can even get more granular and run many more time periods and weights.
### But the number of elements in each list must be the same! And the weights must sum to 100%
| Interval  | Weight |
|-----------|--------|
| Month 1   | 5%     |
| Month 2   | 5%     |
| Month 3   | 5%     |
| Month 4   | 5%     |
| Month 5   | 5%     |
| Month 6   | 5%     |
| Month 7   | 5%     |
| Month 8   | 5%     |
| Month 9   | 5%     |
| Month 10  | 5%     |
| Month 11  | 5%     |
| Month 12  | 5%     |
| Month 18  | 4%     |
| Year 2    | 4%     |
| Year 3    | 4%     |
| Year 4    | 4%     |
| Year 5    | 4%     |
| Year 6    | 4%     |
| Year 7    | 4%     |
| Year 8    | 4%     |
| Year 9    | 4%     |
| Year 10   | 4%     |


In [14]:
periods = [1/12, 2/12, 3/12, 4/12, 5/12, 6/12, 
           7/12, 8/12, 9/12, 10/12, 11/12, 1, 
           1.5, 2, 3, 4, 5, 6, 7, 8, 9, 10]

weights = [0.05]*12 + [0.04]*10
           
my_symbol = "PFE"

wgt_avg_min = wgt_avg_min_close_price(my_symbol, periods, weights)
    
print(f"\nWeighted Average Minimum Close Price for {my_symbol} Stock: ${wgt_avg_min:.2f}")


Weighted Average Minimum Close Price for PFE Stock: $21.18


#### As you can observe, the original weighted average minimum was 21.74
#### But after adding more time periods and weights, it decreased by more than 50 cents to 21.18
#### Furthermore, there seems to be a VERY strong support level at 20.83 (e.g. basically a strike of 21)

#### Now I can be reasonably certain the stock won't fall below 21
#### Even though it was recently that low, that's a historical low
#### The current price is around 24 per share. 
#### So 22-23 might be a pretty safe strike price to sell puts, given the floor of 21 per share

# Why This Matters
#### If I'm writing a put option, I want to be fairly certain my selected strike price is reasonably low to collect premium
#### But not so low that I get assigned (e.g. I'm forced to buy the shares below market value for an unrealized loss)
#### This is one method of technical analysis that models the minimum 'floor' price based on actual data 
#### (We're using actual close prices and analyzing thousands of entries, e.g. we're not just guessing by looking at charts and visuals)
#### Taken alone it may be a weak indicator, but when combined with other data, it can help you make better data-driven decisions
#### There are many other uses for this type of output. I'm sharing how I leverage this sort of data analysis in my own day to day trading

# Now, let's find the Weighted Average Minimum Price for Multiple Stocks

In [15]:
periods = [1/12, 2/12, 3/12, 4/12, 5/12, 6/12, 
           7/12, 8/12, 9/12, 10/12, 11/12, 1, 
           1.5, 2, 3, 4, 5, 6, 7, 8, 9, 10]

weights = [0.05]*12 + [0.04]*10

symbols = ['PFE', 'VZ', 'INTC', 'AAPL', 'GOOGL', 'AMZN', 'NKE', 'TGT',
           'CPB', 'GIS', 'SBUX', 'MCD', 'JNJ', 'MMM', 'SPY', 'IEI']

min_prices = {}

for symbol in symbols:
    min_prices[symbol] = wgt_avg_min_close_price(symbol, periods, weights)

# Only show two decimal places in the dataframe
pd.options.display.float_format = '{:.2f}'.format 

df = pd.DataFrame(list(min_prices.items()), columns=['Symbol', 'Avg Min'])
df

,Symbol,Avg Min
0,PFE,21.18
1,VZ,34.92
2,INTC,18.66
3,AAPL,145.09
4,GOOGL,123.94
5,AMZN,141.43
6,NKE,54.69
7,TGT,79.40
8,CPB,29.68
9,GIS,44.93


# Let's add a column for current market price, to compare

In [16]:
symbols = ['PFE', 'VZ', 'INTC', 'AAPL', 'GOOGL', 'AMZN', 'NKE', 'TGT',
           'CPB', 'GIS', 'SBUX', 'MCD', 'JNJ', 'MMM', 'SPY', 'IEI']

mkt_prices = []

for symbol in symbols:
    mkt_prices.append(yf.Ticker(symbol).info.get('currentPrice'))

mkt_prices

[23.76,
 43.61,
 35.5,
 255.46,
 246.54,
 219.78,
 69.31,
 87.85,
 32.08,
 50.09,
 83.39,
 305.24,
 179.71,
 152.81,
 None,
 None]

#### Don't worry about the 'None's for now. We can always handle that later. Those are ETFs anyways. 
#### Now let's calculate the percentage distance the current price is relative to the wgt avg min we calculated
#### The formula is Minimum Price / Market Price - 1. This tells us the % move the stock must make to reach the minimum price
#### For example if the Market Price is 100 and the Wgt Avg Min Price is 90...
#### The price would need to move 90/100 - 1 = -10% 
#### The price would have to move down 10% for the Market Price to equal the Minimum Price
#### Proof: 100 * (100% - 10%) = 100 * 90% = 90

# Now let's add the market prices and the '% to Min' column to the dataframe!

In [17]:
df['Current Price'] = mkt_prices
df

,Symbol,Avg Min,Current Price
0,PFE,21.18,23.76
1,VZ,34.92,43.61
2,INTC,18.66,35.50
3,AAPL,145.09,255.46
4,GOOGL,123.94,246.54
5,AMZN,141.43,219.78
6,NKE,54.69,69.31
7,TGT,79.40,87.85
8,CPB,29.68,32.08
9,GIS,44.93,50.09


In [18]:
df['% to Min'] = (df['Avg Min'] / df['Current Price']) - 1
df

,Symbol,Avg Min,Current Price,% to Min
0,PFE,21.18,23.76,-0.11
1,VZ,34.92,43.61,-0.20
2,INTC,18.66,35.50,-0.47
3,AAPL,145.09,255.46,-0.43
4,GOOGL,123.94,246.54,-0.50
5,AMZN,141.43,219.78,-0.36
6,NKE,54.69,69.31,-0.21
7,TGT,79.40,87.85,-0.10
8,CPB,29.68,32.08,-0.07
9,GIS,44.93,50.09,-0.10


#### And there you go. You can quickly see which stocks are way above their historical support levels 
#### For example INTC, AAPL, GOOGL would all need to decline more than 40% to reach the weighted average minimum
#### And you can also see which stocks are near historical lows. For example PFE, CPB, TGT, GIS
#### These stocks would only need to deline 7-10% to reach historically low levels (i.e. the support level)

# As you can see, it's quick, easy, and useful to do technical analysis in Python!